In [ ]:
from glob import glob
from polyphys.manage import organizer

import MDAnalysis as mda
import numpy as np
from polyphys.manage.parser import (
    SumRuleCyl, TransFociCyl, TransFociCub, HnsCub, HnsCyl
    )
from polyphys.manage.typer import ParserT

import warnings

In [ ]:
def write_lammps_data(
    topology: str,
    trajectory: str,
    sim_info: ParserT,
    timestep_offset: int = 0,
    write_initial: int = 0,
    write_interval: int = 1000,
    save_to: str = './',
) -> None:
    """
    Write a series of data files from a molecular dynamics simulation. Each file
    corresponds to a specific timestep in the trajectory.

    Parameters
    ----------
    topology: str
        Name of the topology file.
    trajectory: str
        Name of the trajectory file.
    sim_info: ParserT
        Simulation information parser.
    timestep_offset: int, default 0
        The initial time step of the trajectory file.
    write_initial: int, default 0
        The initial timesteps form which data files are created.
    write_interval: int, default 1000
        The frequency (in timesteps) at which data files are created.
    save_to: str, default './'
        The path where output files are saved.
    """
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print("\n" + sim_name + " is analyzing...\n")
    cell = mda.Universe(
        topology, trajectory, topology_format='DATA',
        format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
        atom_style="id resid type x y z")
    # slicing trajectory based the continuous condition
    
    sliced_trj = cell.trajectory[write_initial::write_interval]
    # selecting atom groups
    bug: mda.AtomGroup = cell.select_atoms('resid 1')  # the polymer
    for _ in sliced_trj:
        # various measures of chain size
        frame = cell.trajectory.frame
        frame_corrected = \
            timestep_offset + (frame) * sim_info.bdump
        center_of_mass = bug.center_of_mass()
        # Translate the atom group to recenter it at the origin
        bug.translate(-center_of_mass)
        # Calculate the new box dimensions
        #max_extent = np.max(np.abs(bug.positions), axis=0)
        #new_box_dimensions = 2 * max_extent + 1
        # Manually set the universe dimensions
        #cell.dimensions[:3] = new_box_dimensions
        new_box_dimensions = cell.dimensions[:3]
        outfile = save_to + sim_name + f"-timestep{frame_corrected}.data"
        bug.write(outfile, format='data')
        # Now read the written file and adjust the box dimensions
        with open(outfile, 'r') as file:
            lines = file.readlines()

        with open(outfile, 'w') as file:
            for line in lines:
                if 'xlo xhi' in line:
                    line = f"{-new_box_dimensions[0]/2} {new_box_dimensions[0]/2} xlo xhi\n"
                elif 'ylo yhi' in line:
                    line = f"{-new_box_dimensions[1]/2} {new_box_dimensions[1]/2} ylo yhi\n"
                elif 'zlo zhi' in line:
                    line = f"{-new_box_dimensions[2]/2} {new_box_dimensions[2]/2} zlo zhi\n"
                file.write(line)
    print('done.')


# analyzing bug files.

lineage = 'whole'
geometry = 'cylindrical'
group = 'bug'
polymer_topology = 'linear'

save_to = '/Users/amirhsi_mini/research_data/sumrule/'
database = '/Users/amirhsi_mini/research_data/sumrule/'
#database = './'
hierarchy = 'N*'
bug_pairs = glob(database + hierarchy + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
print(bug_pairs)
for (bug_topo, bug_trj) in bug_pairs:
    print(bug_topo)
    sim_info = SumRuleCyl(
            bug_trj,
            lineage,
            geometry,
            group,
            polymer_topology,
        )
    write_lammps_data(
        bug_topo,
        bug_trj,
        sim_info,
        timestep_offset=1000000,
        write_initial=60000,
        write_interval=200,
        save_to=save_to
    )

In [ ]:
# finding the real timesteps in a LAMMPS trajecotry file.
with open("/Users/amirhsi_mini/research_data/sumrule/N2000epsilon5.0r30.5lz233.5sig4.0nc0dt0.01bdump5000adump10000ens1.bug.lammpstrj", 'r') as f:
    with open('/Users/amirhsi_mini/research_data/sumrule/timesteps.txt', 'w') as dest_file:
        # Initialize a line counter
        counter = 1

        # Iterate over each line in the file
        for line in f:
            # Check if the counter is a multiple of 2009
            if counter > (60000*2009):
                if (counter - 2) % 2009 == 0:
                    # Process the line (e.g., print it)
                    dest_file.write(line)
                # Increment the line counter
            counter += 1
